In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean

In [ ]:
# Estadios con la dirección polar latitud y longitud
"""
Ciudad	                Estado	                Estadio	                Capacidad	latitud	    longitud
Atlanta	                Georgia	                Mercedes-Benz Stadium   71.000 	    33.755489	-84.401993
Boston	                Massachuts	            Gillette Stadium	    68.756 	    42.0922 	-71.2642
Dallas	                Texas	                AT&T Stadium	        80.000 	    32.747841	-97.093628
Houston	                Texas	                NRG Stadium	            72.220 	    29.684860	-95.411667
Kansas City	            Misuri	                Arrowhead Stadium	    76.416 	    39.048786	-94.484566
Los Angeles	            California	            SoFi Stadium	        70.000 	    33.953587	-118.339630
Miami	                Florida                 Hard Rock Stadium	    65.326 	    25.957960	-80.239311
New York/New Jersey     New York/New Jersey     MetLife Stadium	        82.500 	    40.813778	-74.074310
Philadelfia	            Pensilvania	            Lincoln Financial Field 67.594 	    39.900898	-75.168098
San Francisco	        California	            Levi's Stadium	        68.500 	    37.4036	    -121.9702
Seatle	                Washinton	            Lumen Field     	    72.000 	    47.595097	-122.332245
"""

Creamos funcion:

In [3]:
def encontrar_sitios_cercanos(df_cercanos, df_metadata_sitios, punto1, distancia1):
    # Iterar sobre los registros del DataFrame original
    for index, row in df_metadata_sitios.iterrows():
        latitud = row['latitude']
        longitud = row['longitude']
        punto2 = (latitud, longitud)
        # Calcular la distancia entre el punto1 y el punto2
        distancia = euclidean(punto1, punto2) * 100
        # Verificar si la distancia es menor o igual a 50 km
        if distancia <= distancia1:
            # Almacenar el registro completo en el nuevo DataFrame
            df_cercanos.loc[len(df_cercanos)] = row
    return df_cercanos


Leer dataset meda:

In [4]:
metada_sitios = pd.read_parquet('../Dataset Final Google/Metadata Sitios/metadata_sitios.parquet')

Borramos columna:

In [5]:
metada_sitios.drop(columns= ['url','relative_results', 'MISC', 'description', 'price','hours'], inplace = True)

Desanidamos:

In [6]:
metadata_sitios_final = metada_sitios.explode(column= "category") # creamos un dataframe donde duplica,os las filas de acuerdo a la cantidad de subcampos que hay en la columna review.


Borramos duplicados por gmap_id y reseteamos:

In [7]:
metadata_sitios_final.drop_duplicates(inplace = True)

In [8]:
metadata_sitios_final.reset_index(inplace=True, drop=True) # Reseteamos los indices del dataframe desanidado para poder concatenar

Guardamos:

In [ ]:
metadata_sitios_final.to_parquet('../Estadios_reviews/metadata_sitios_final.parquet', compression= 'Gzip', index = False)

## Arrowhead Stadium - MISSOURI y KANSAS

Leemos datasets

In [ ]:
#Cargar el conjunto de datos de negocios filtrados
missouri=pd.read_parquet('../Dataset Final Google/Reviews Estados/Missouri.parquet')

In [ ]:
#Cargar el conjunto de datos de negocios filtrados
kansas=pd.read_parquet('../Dataset Final Google/Reviews Estados/Kansas.parquet')

In [ ]:
#Combinamos ambos df
missouri_kansas = pd.concat([missouri, kansas], ignore_index=True)

Transformamos y borramos columnas:

In [ ]:

#Eliminamos las columnas que no necesaritaremos y la que contengan muchos valores nulos
missouri_kansas.drop(columns=['pics' ,'resp','name'], inplace=True)

In [ ]:
# missouri_kansas['time'] = pd.to_datetime(missouri_kansas['time'], unit='ms')

In [ ]:
#Modificamos los reviews faltante por Neutral
missouri_kansas['text'].fillna('Neutral', inplace=True)

In [ ]:
missouri_kansas.drop_duplicates(inplace=True)

In [ ]:
missouri_kansas.to_parquet('../Estadios_reviews/missouri_kansas.parquet',compression='gzip', index=False)

In [ ]:
punto_missouri_kansas = (39.048786, -94.484566) # Establecemos el estadio: Arrowhead Stadium - Kansas City - Estado de Misuri

distancia = 50 # Distancia 50 kilometros cerca del estadio

# Crea un DataFrame vacío para almacenar los registros que cumplen con la condición
df_cercanos_missouri_kansas = pd.DataFrame(columns=metadata_sitios_final.columns)  # Asumiendo que df es el DataFrame original

#Llama la función para encontrar todos los puntos cercanos a la distancia seleccionada
df_cercanos_missouri_kansas = encontrar_sitios_cercanos(df_cercanos_missouri_kansas, metadata_sitios_final, punto_missouri_kansas, distancia)

In [ ]:
#Filtra el resultado de la metadata con el dataset del estado
df_filtrado__missouri_kansas = pd.merge(missouri_kansas, df_cercanos_missouri_kansas, on='gmap_id')

In [ ]:
df_filtrado__missouri_kansas.to_parquet('../Datasets_final/Arrowhead_Stadium.parquet', compression='gzip',index=False)

## MetLife Stadium- NEW JERSEY y NEW YORK:

Leemos los df

In [ ]:
New_Jersey = pd.read_parquet('../Dataset Final Google/Reviews Estados/New_Jersey.parquet')

In [ ]:
New_York = pd.read_parquet('../Dataset Final Google/Reviews Estados/New_York.parquet')

In [ ]:
new_jersey_new_york = pd.concat([New_Jersey, New_York], ignore_index=True)

Transformamos:

In [ ]:

#Eliminamos las columnas que no necesaritaremos y la que contengan muchos valores nulos
new_jersey_new_york.drop(columns=['pics' ,'resp', 'name'], inplace=True)

In [ ]:
# new_jersey_new_york['time'] = pd.to_datetime(new_jersey_new_york['time'], unit='ms')

In [ ]:
#Modificamos los reviews faltante por Neutral
new_jersey_new_york['text'].fillna('Neutral', inplace=True)

In [ ]:
new_jersey_new_york.drop_duplicates(inplace=True)

In [ ]:
new_jersey_new_york.to_parquet('../Estadios_reviews/new_jersey_new_york.parquet',compression='gzip', index=False)

Ejecutamos la funcion:

In [ ]:
punto1 = (40.813778, -74.074310) #MetLife Stadium - New York/New Jersey - Estados de New York/New Jersey

distancia = 50 # Distancia 30 kilometros cerca del estadio 

# Crea un DataFrame vacío para almacenar los registros que cumplen con la condición
df_cercanos_new_jersey_new_york = pd.DataFrame(columns=metadata_sitios_final.columns)  # Asumiendo que df es el DataFrame original

#Llama la función para encontrar todos los puntos cercanos a la distancia seleccionada
df_cercanos_new_jersey_new_york = encontrar_sitios_cercanos(df_cercanos_new_jersey_new_york, metadata_sitios_final, punto1, distancia)

Unimos con los datasets estados:

In [ ]:
#Filtra el resultado de la metadata con el dataset del estado
df_filtrado_new_jersey_new_york = pd.merge(new_jersey_new_york, df_cercanos_new_jersey_new_york, on='gmap_id')

Guardamos:

In [ ]:
df_filtrado_new_jersey_new_york.to_parquet('../Datasets_final/MetLife_Stadium.parquet', compression='gzip',index=False)

## Lincoln Financial Field - PENSILVANIA:

Leemos:

In [9]:
pensilvania = pd.read_parquet('../Dataset Final Google/Reviews Estados/Pensilvania.parquet')

Transformamos:

In [10]:

#Eliminamos las columnas que no necesaritaremos y la que contengan muchos valores nulos
pensilvania.drop(columns=['pics' ,'resp', 'name'], inplace=True)

In [ ]:
# pensilvania['time'] = pd.to_datetime(pensilvania['time'], unit='ms')

In [11]:
#Modificamos los reviews faltante por Neutral
pensilvania['text'].fillna('Neutral', inplace=True)

C:\Users\patri\AppData\Local\Temp\ipykernel_6164\3116476257.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  pensilvania['text'].fillna('Neutral', inplace=True)


In [12]:
pensilvania.drop_duplicates(inplace=True)

In [ ]:
pensilvania.to_parquet('../Estadios_reviews/pensilvania.parquet',compression='gzip', index=False)

Ejecutamos funcion:

In [13]:
punto1 = (39.900898,-75.168098) #Lincoln Financial Field - Philadelfia - Estado de Pensilvania

distancia = 50 # Distancia 30 kilometros cerca del estadio 

# Crea un DataFrame vacío para almacenar los registros que cumplen con la condición
df_cercanos_pensilvania = pd.DataFrame(columns=metadata_sitios_final.columns)  # Asumiendo que df es el DataFrame original

#Llama la función para encontrar todos los puntos cercanos a la distancia seleccionada
df_cercanos_pensilvania = encontrar_sitios_cercanos(df_cercanos_pensilvania, metadata_sitios_final, punto1, distancia)

Unimos: 

In [16]:
#Filtra el resultado de la metadata con el dataset de reviews del estado respectivo
df_filtrado_pensilvania = pd.merge(pensilvania, df_cercanos_pensilvania, on='gmap_id')

Guardamos:

In [17]:
df_filtrado_pensilvania.to_parquet('../Datasets_final/Lincoln_Financial_Field.parquet', compression='gzip',index=False)

## Gillete Stadium - MASSACHUSETTS:

Leemos:

In [18]:
massachusetts = pd.read_parquet('../Dataset Final Google/Reviews Estados/massachusetts.parquet') 

Transformamos:

In [19]:
#Eliminamos las columnas que no necesaritaremos y la que contengan muchos valores nulos
massachusetts.drop(columns=['pics' ,'resp', 'name'], inplace=True)

In [ ]:
# massachusetts['time'] = pd.to_datetime(massachusetts['time'], unit='ms')

In [20]:
#Modificamos los reviews faltante por Neutral
massachusetts['text'].fillna('Neutral', inplace=True)

C:\Users\patri\AppData\Local\Temp\ipykernel_6164\3944610266.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  massachusetts['text'].fillna('Neutral', inplace=True)


In [21]:
massachusetts.drop_duplicates(inplace=True)

In [ ]:
massachusetts.to_parquet('../Estadios_reviews/massachusetts.parquet',compression='gzip', index=False)

Usamos la funcion:

In [22]:
punto1 = (42.0922, -71.2642) # Gillete Stadium - Boston - Estado de Massachutts

distancia = 50 # Distancia 30 kilometros cerca del estadio 

# Crea un DataFrame vacío para almacenar los registros que cumplen con la condición
df_cercanos_massachusetts = pd.DataFrame(columns=metadata_sitios_final.columns)  # Asumiendo que df es el DataFrame original

#Llama la función para encontrar todos los puntos cercanos a la distancia seleccionada
df_cercanos_massachusetts = encontrar_sitios_cercanos(df_cercanos_massachusetts, metadata_sitios_final, punto1, distancia)

Unimos:

In [23]:
#Filtra el resultado de la metadata con el dataset de reviews del estado respectivo
df_cercanos_massachusetts = pd.merge(massachusetts, df_cercanos_massachusetts, on='gmap_id')

Guardamos:

In [24]:
df_cercanos_massachusetts.to_parquet('../Datasets_final/Gillete_Stadium.parquet', compression='gzip',index=False)

## SoFi Stadium y LEVI´S STADIUM - CALIFORNIA:

Leer datasets:

In [25]:
#Cargar el conjunto de datos de negocios filtrados 
california = pd.read_parquet('../Dataset Final Google/Reviews Estados/California.parquet')

Transformacion:

In [26]:
#Eliminamos las columnas que no necesaritaremos y la que contengan muchos valores nulos
california.drop(columns=['pics' ,'resp', 'name'], inplace=True)

In [ ]:
# california['time'] = pd.to_datetime(california['time'], unit='ms')

In [27]:
#Modificamos los reviews faltante por Neutral
california['text'].fillna('Neutral', inplace=True)

C:\Users\patri\AppData\Local\Temp\ipykernel_6164\3185630916.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  california['text'].fillna('Neutral', inplace=True)


In [28]:
california.drop_duplicates(inplace=True)

In [ ]:
#Guardamos la data limpia
california.to_parquet('../Estadios_reviews/california.parquet',compression='gzip', index=False)

Aplicamos funcion:

In [31]:
#Ubicacion del estadio SOFI STADIUM
punto1 = (33.953587, -118.339630) #SoFi Stadium - Los Angeles - Estado de California
#Distancia a la redonda del estadio donde se quiere ubicar los puntos, dada en kilómetros
distancia = 50
#Crea un DataFrame vacío para almacenar los registros que cumplen con la condición
df_cercanos_california = pd.DataFrame(columns=metadata_sitios_final.columns)  # Asumiendo que df es el DataFrame original
#Llama la función para encontrar todos los puntos cercanos a la distancia seleccionada
df_cercanos_california = encontrar_sitios_cercanos(df_cercanos_california, metadata_sitios_final, punto1, distancia)

Unimos:

In [32]:
#Filtra el resultado de la metadata con el dataset del estado
df_filtrado_california = pd.merge(california, df_cercanos_california, on='gmap_id')

Guardamos:

In [33]:
df_filtrado_california.to_parquet('../Datasets_final/SoFi_Stadium.parquet', compression='gzip',index=False)

Aplicacion Funcion 2:

In [34]:
#Ubicacion del estadio LEVI´S STADIUM
punto1 = (37.4036, -121.9702) #Levi's Stadium - San Francisco - Estado de California
#Distancia a la redonda del estadio donde se quiere ubicar los puntos, dada en kilómetros
distancia = 50
#Crea un DataFrame vacío para almacenar los registros que cumplen con la condición
df_cercanos_california2 = pd.DataFrame(columns=metadata_sitios_final.columns)  # Asumiendo que df es el DataFrame original
#Llama la función para encontrar todos los puntos cercanos a la distancia seleccionada
df_cercanos_california2 = encontrar_sitios_cercanos(df_cercanos_california2, metadata_sitios_final, punto1, distancia)

In [35]:
#Filtra el resultado de la metadata con el dataset del estado
df_filtrado_california2 = pd.merge(california, df_cercanos_california2, on='gmap_id')

In [36]:
df_filtrado_california2.to_parquet('../Datasets_final/LEVI´S_STADIUM.parquet', compression='gzip',index=False)

## HARD ROCK STADIUM - FLORIDA

Leer df:

In [37]:
#Cargar el conjunto de datos de negocios filtrados 
florida = pd.read_parquet('../Dataset Final Google/Reviews Estados/Florida.parquet')

Transformamos:

In [38]:
#Eliminamos las columnas que no necesaritaremos y la que contengan muchos valores nulos
florida.drop(columns=['pics' ,'resp', 'name'], inplace=True)

In [ ]:
# florida['time'] = pd.to_datetime (florida['time'], unit='ms')

In [39]:
#Modificamos los reviews faltante por Neutral
florida['text'].fillna('Neutral', inplace=True)

C:\Users\patri\AppData\Local\Temp\ipykernel_6164\4070066519.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  florida['text'].fillna('Neutral', inplace=True)


In [40]:
florida.drop_duplicates(inplace=True)

Guardamos:

In [ ]:
#Guardamos la data limpia
florida.to_parquet('../Estadios_reviews/florida.parquet',compression='gzip', index=False)

Aplicacion funcion:

In [42]:
#Ubicacion del estadio HARD ROCK STADIUM
punto1 = (25.957960, -80.239311) #Hard Rock Stadium - Miami - Estado de La Florida
#Distancia a la redonda del estadio donde se quiere ubicar los puntos, dada en kilómetros
distancia = 50
#Crea un DataFrame vacío para almacenar los registros que cumplen con la condición
df_cercanos_florida = pd.DataFrame(columns=metadata_sitios_final.columns)  # Asumiendo que df es el DataFrame original
#Llama la función para encontrar todos los puntos cercanos a la distancia seleccionada
df_cercanos_florida = encontrar_sitios_cercanos(df_cercanos_florida, metadata_sitios_final, punto1, distancia)

Unimos:

In [43]:
#Filtra el resultado de la metadata con el dataset del estado
df_filtrado_florida = pd.merge(florida, df_cercanos_florida, on='gmap_id')

Guardamos:

In [44]:
df_filtrado_florida.to_parquet('../Datasets_final/HARD_ROCK_STADIUM.parquet', compression='gzip',index=False)

## Mercedes-Benz - GEORGIA

Leer:

In [45]:
#Cargar el conjunto de datos de negocios filtrados 
georgia = pd.read_parquet('../Dataset Final Google/Reviews Estados/georgia.parquet')

Transformacion:

In [46]:
#Eliminamos las columnas que no necesaritaremos y la que contengan muchos valores nulos
georgia.drop(columns=['pics' ,'resp', 'name'], inplace=True)

In [ ]:
# georgia['time'] = pd.to_datetime(georgia['time'],unit='ms')

In [47]:
#Modificamos los reviews faltante por Neutral
georgia['text'].fillna('Neutral', inplace=True)

C:\Users\patri\AppData\Local\Temp\ipykernel_6164\155885781.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  georgia['text'].fillna('Neutral', inplace=True)


In [48]:
georgia.drop_duplicates(inplace=True)

In [ ]:
#Guardamos la data limpia
georgia.to_parquet('../Estadios_reviews/georgia.parquet',compression='gzip', index=False)

In [49]:
#Ubicacion del estadio
punto1 = (33.755489, -84.401993) #Mercedes-Benz Stadium - Atlanta - Estado de Georgia
#Distancia a la redonda del estadio donde se quiere ubicar los puntos, dada en kilómetros
distancia = 50
#Crea un DataFrame vacío para almacenar los registros que cumplen con la condición
df_cercanos_georgia = pd.DataFrame(columns=metadata_sitios_final.columns)  # Asumiendo que df es el DataFrame original
#Llama la función para encontrar todos los puntos cercanos a la distancia seleccionada
df_cercanos_georgia = encontrar_sitios_cercanos(df_cercanos_georgia, metadata_sitios_final, punto1, distancia)

Unimos:

In [50]:
#Filtra el resultado de la metadata con el dataset de reviews del estado
df_filtrado_georgia = pd.merge(georgia, df_cercanos_georgia, on='gmap_id')

Guardamos:

In [51]:
df_filtrado_georgia.to_parquet('../Datasets_final/Mercedes_Benz.parquet', compression='gzip',index=False)

## AT&T Stadium y NRG Stadum - TEXAS:

Leer:

In [52]:
#Cargar el conjunto de datos de negocios filtrados 
texas = pd.read_parquet('../Dataset Final Google/Reviews Estados/Texas.parquet')

Transformamos:

In [53]:
#Eliminamos las columnas que no necesaritaremos y la que contengan muchos valores nulos
texas.drop(columns=['pics' ,'resp', 'name'], inplace=True)

In [ ]:
# texas['time'] = pd.to_datetime(texas['time'], unit='ms')

In [54]:
#Modificamos los reviews faltante por Neutral
texas['text'].fillna('Neutral', inplace=True)

C:\Users\patri\AppData\Local\Temp\ipykernel_6164\615764583.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  texas['text'].fillna('Neutral', inplace=True)


In [55]:
texas.drop_duplicates(inplace=True)

In [ ]:
#Guardamos la data limpia
texas.to_parquet('../Estadios_reviews/texas.parquet',compression='gzip', index=False)

Aplicamos Funcion:

In [56]:
#Ubicacion del estadio
punto1 = (32.747841, -97.093628) #AT&T Stadium - Dallas - Estado de Texas
#Distancia a la redonda del estadio donde se quiere ubicar los puntos, dada en kilómetros
distancia = 50
#Crea un DataFrame vacío para almacenar los registros que cumplen con la condición
df_cercanos_texas = pd.DataFrame(columns= metadata_sitios_final.columns)  # Asumiendo que df es el DataFrame original
#Llama la función para encontrar todos los puntos cercanos a la distancia seleccionada
df_cercanos_texas = encontrar_sitios_cercanos(df_cercanos_texas, metadata_sitios_final, punto1, distancia)

In [57]:
#Filtra el resultado de la metadata con el dataset del estado
df_filtrado_texas = pd.merge(texas, df_cercanos_texas, on='gmap_id')

In [58]:
df_filtrado_texas.to_parquet('../Datasets_final/AT&T_Stadium.parquet', compression='gzip',index=False)

Aplicar funcion 2:

In [59]:
#Ubicacion del estadio
punto1 = (29.684860, -95.411667) #NRG Stadum - Houston - Estado de Texas
#Distancia a la redonda del estadio donde se quiere ubicar los puntos, dada en kilómetros
distancia = 50
#Crea un DataFrame vacío para almacenar los registros que cumplen con la condición
df_cercanos_texas2 = pd.DataFrame(columns=metadata_sitios_final.columns)  # Asumiendo que df es el DataFrame original
#Llama la función para encontrar todos los puntos cercanos a la distancia seleccionada
df_cercanos_texas2 = encontrar_sitios_cercanos(df_cercanos_texas2, metadata_sitios_final, punto1, distancia)

In [60]:
#Filtra el resultado de la metadata con el dataset del estado
df_filtrado_texas2 = pd.merge(texas, df_cercanos_texas2, on='gmap_id')

In [61]:
df_filtrado_texas2.to_parquet('../Datasets_final/NRG_Stadum.parquet', compression='gzip',index=False)

## Lumen Field - WASHINGTON:

Leer:

In [62]:
#Cargar el conjunto de datos de negocios filtrados 
washington = pd.read_parquet('../Dataset Final Google/Reviews Estados/Washington.parquet')

Transformamos:

In [63]:
#Eliminamos las columnas que no necesaritaremos y la que contengan muchos valores nulos
washington.drop(columns=['pics' ,'resp', 'name'], inplace=True)

In [ ]:
# washington['time'] = pd.to_datetime(washington['time'], unit='ms')

In [64]:
#Modificamos los reviews faltante por Neutral
washington['text'].fillna('Neutral', inplace=True)

C:\Users\patri\AppData\Local\Temp\ipykernel_6164\2831210142.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  washington['text'].fillna('Neutral', inplace=True)


In [65]:
washington.drop_duplicates(inplace=True)

In [ ]:
#Guardamos la data limpia
washington.to_parquet('../Estadios_reviews/washington.parquet',compression='gzip', index=False)

Aplicamos funciones:

In [66]:
#Ubicacion del estadio
punto1 = (47.595097, -122.332245) #Lumen Field latitud - Seatle - Estado de Washington
#Distancia a la redonda del estadio donde se quiere ubicar los puntos, dada en kilómetros
distancia = 50
#Crea un DataFrame vacío para almacenar los registros que cumplen con la condición
df_cercanos_washington = pd.DataFrame(columns=metadata_sitios_final.columns)  # Asumiendo que df es el DataFrame original
#Llama la función para encontrar todos los puntos cercanos a la distancia seleccionada
df_cercanos_washington = encontrar_sitios_cercanos(df_cercanos_washington, metadata_sitios_final, punto1, distancia)

In [67]:
#Filtra el resultado de la metadata con el dataset del estado
df_filtrado_washington = pd.merge(washington, df_cercanos_washington, on='gmap_id')

In [68]:
df_filtrado_washington.to_parquet('../Datasets_final/Lumen_Field.parquet', compression='gzip',index=False)

# Combinamos los dataframes finales

In [69]:
Arrowhead_Stadium = pd.read_parquet('../Datasets_final/Arrowhead_Stadium.parquet')
ATyT_Stadium = pd.read_parquet('../Datasets_final/AT&T_Stadium.parquet')
Gillete_Stadium = pd.read_parquet('../Datasets_final/Gillete_Stadium.parquet')
HARD_ROCK_STADIUM = pd.read_parquet('../Datasets_final/HARD_ROCK_STADIUM.parquet')
LEVI_STADIUM = pd.read_parquet('../Datasets_final/LEVI´S_STADIUM.parquet')
Lincoln_Financial_Field = pd.read_parquet('../Datasets_final/Lincoln_Financial_Field.parquet')
Lumen_Field = pd.read_parquet('../Datasets_final/Lumen_Field.parquet')
Mercedes_Benz = pd.read_parquet('../Datasets_final/Mercedes_Benz.parquet')
MetLife_Stadium = pd.read_parquet('../Datasets_final/MetLife_Stadium.parquet')
NRG_Stadum = pd.read_parquet('../Datasets_final/NRG_Stadum.parquet')
SoFi_Stadium = pd.read_parquet('../Datasets_final/SoFi_Stadium.parquet')


In [ ]:
# Estadios con la dirección polar latitud y longitud
"""
Ciudad	                Estado	                Estadio	                Capacidad	latitud	    longitud
Atlanta	                Georgia	                Mercedes-Benz Stadium   71.000 	    33.755489	-84.401993
Boston	                Massachuts	            Gillette Stadium	    68.756 	    42.0922 	-71.2642
Dallas	                Texas	                AT&T Stadium	        80.000 	    32.747841	-97.093628
Houston	                Texas	                NRG Stadum	            72.220 	    29.684860	-95.411667
Kansas City	            Misuri	                Arrowhead Stadium	    76.416 	    39.048786	-94.484566
Los Angeles	            California	            SoFi Stadium	        70.000 	    33.953587	-118.339630
Miami	                Florida                 Hard Rock Stadium	    65.326 	    25.957960	-80.239311
New York/New Jersey     New York/New Jersey     MetLife Stadium	        82.500 	    40.813778	-74.074310
Philadelfia	            Pensilvania	            Lincoln Financial Field 67.594 	    39.900898	-75.168098
San Francisco	        California	            Levi's Stadium	        68.500 	    37.4036	    -121.9702
Seatle	                Washinton	            Lumen Field     	    72.000 	    47.595097	-122.332245
"""

### Agregar columnas nuevas

In [73]:
Arrowhead_Stadium = pd.read_parquet('../Datasets_final/Arrowhead_Stadium.parquet')
Arrowhead_Stadium['Estadio cercano'] = 'Arrowhead Stadium'
Arrowhead_Stadium['Ciudad'] = 'Kansas City'
Arrowhead_Stadium['Estado'] = 'Misuri'

In [74]:
ATyT_Stadium = pd.read_parquet('../Datasets_final/AT&T_Stadium.parquet')
ATyT_Stadium['Estadio cercano'] = 'AT&T Stadium'
ATyT_Stadium['Ciudad'] = 'Dallas'
ATyT_Stadium['Estado'] = 'Texas'


In [75]:
Gillete_Stadium = pd.read_parquet('../Datasets_final/Gillete_Stadium.parquet')
Gillete_Stadium['Estadio cercano'] = 'Gillete Stadium'
Gillete_Stadium['Ciudad'] = 'Boston'
Gillete_Stadium['Estado'] = 'Massachusetts'

In [85]:
HARD_ROCK_STADIUM = pd.read_parquet('../Datasets_final/HARD_ROCK_STADIUM.parquet')
HARD_ROCK_STADIUM['Estadio cercano'] = 'Hard Rock Stadium'
HARD_ROCK_STADIUM['Ciudad'] = 'Miami'
HARD_ROCK_STADIUM['Estado'] = 'Florida'

In [84]:
LEVI_STADIUM = pd.read_parquet('../Datasets_final/LEVI´S_STADIUM.parquet')
LEVI_STADIUM['Estadio cercano'] = 'Levi´s Stadium'
LEVI_STADIUM['Ciudad'] = 'San Francisco'
LEVI_STADIUM['Estado'] = 'California'

In [78]:
Lincoln_Financial_Field = pd.read_parquet('../Datasets_final/Lincoln_Financial_Field.parquet')
Lincoln_Financial_Field['Estadio cercano'] = 'Lincoln Financial Field'
Lincoln_Financial_Field['Ciudad'] = 'Philadelfia'
Lincoln_Financial_Field['Estado'] = 'Pennsylvania'

In [79]:
Lumen_Field = pd.read_parquet('../Datasets_final/Lumen_Field.parquet')
Lumen_Field['Estadio cercano'] = 'Lumen Field'
Lumen_Field['Ciudad'] = 'Sattle'
Lumen_Field['Estado'] = 'Washinton'

In [80]:
Mercedes_Benz = pd.read_parquet('../Datasets_final/Mercedes_Benz.parquet')
Mercedes_Benz['Estadio cercano'] = 'Mercedes-Benz Stadium'
Mercedes_Benz['Ciudad'] = 'Atlanta'
Mercedes_Benz['Estado'] = 'Georgia'

In [81]:
MetLife_Stadium = pd.read_parquet('../Datasets_final/MetLife_Stadium.parquet')
MetLife_Stadium['Estadio cercano'] = 'MetLife Stadium'
MetLife_Stadium['Ciudad'] = 'Bergen'
MetLife_Stadium['Estado'] = 'Nueva York/Nueva Jersey'

In [82]:
NRG_Stadum = pd.read_parquet('../Datasets_final/NRG_Stadum.parquet')
NRG_Stadum['Estadio cercano'] = 'NRG Stadium'
NRG_Stadum['Ciudad'] = 'Houston'
NRG_Stadum['Estado'] = 'Texas'

In [83]:
SoFi_Stadium = pd.read_parquet('../Datasets_final/SoFi_Stadium.parquet')
SoFi_Stadium['Estadio cercano'] = 'SoFi Stadium'
SoFi_Stadium['Ciudad'] = 'Los Angeles'
SoFi_Stadium['Estado'] = 'California'

### Unimos y borramos duplicados y guardamos

In [87]:
lista_datasets_estadios = [Arrowhead_Stadium,ATyT_Stadium,Gillete_Stadium,HARD_ROCK_STADIUM,LEVI_STADIUM,Lincoln_Financial_Field,Lumen_Field,Mercedes_Benz,MetLife_Stadium,NRG_Stadum,SoFi_Stadium]

In [88]:
dataset_ultimo_final = pd.concat(lista_datasets_estadios, ignore_index=True)

In [1]:
dataset_ultimo_final.drop_duplicates(inplace=True)

NameError: name 'dataset_ultimo_final' is not defined

In [ ]:
dataset_ultimo_final.info()

In [ ]:
#Modificamos los reviews faltante por Neutral
dataset_ultimo_final['text'].fillna('Neutral', inplace=True)

In [ ]:
dataset_ultimo_final.drop_duplicates(inplace=True)

In [ ]:
dataset_ultimo_final.to_parquet('./dataset_ultimo_final.parquet', compression='gzip', index=False)